In [38]:
import pandas as pd
import numpy as np
path_mapa = r'c:\Users\wesley.oliveira\Desktop\PRODUTOS PARA TRANSFERENCIA.xlsx'
path_8596 = r'c:\WS_OLIVEIRA\BASE_DADOS\8596 - Dados logistico produtos WMS.xlsx'

df_96 = pd.read_excel(path_8596, usecols= ['CODPROD', 'DESCRICAO','OBS2','RUA','PREDIO','APTO'])
df_96['concat'] = df_96['RUA'].astype(str) + "-" + df_96['PREDIO'].astype(str) + "-" + df_96['APTO'].astype(str)
df_mapa = pd.read_excel(path_mapa, sheet_name= 'resalva')

df = df_mapa.merge(df_96, left_on= 'cod', right_on= 'CODPROD', how= 'left')
df = df.drop(columns=['CODPROD', 'DESCRICAO', 'OBS2','RUA_x', 'PREDIO_x', 'NIVEL_B', 'APTO_x'])




c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [39]:
df['validar'] = np.where(df['concat_x'] == df['concat_y'], "ok", "divergente")

display(df.loc[df['validar'] == "divergente"])


,cod,desc,concat_x,RUA_y,PREDIO_y,APTO_y,concat_y,validar
38,465098,BOLINHO BAUDUCCO 40GR DUPLO CHOCOLATE,12-17-101,12.0,72.0,104.0,12-72-104,divergente
44,469182,BISC CORY 100GR RECH HIPOPO CHOCOLATE,2-47-102,NaN,NaN,NaN,NaN,divergente
49,466397,PAO DE FORMA BAUDUCCO 390GR INTEGRAL,3-55-101,70.0,6.0,112.0,70-6-112,divergente
83,452678,COCO RALADO ADEL COCO 1KG ADOC FLOCOS UM,12-72-104,12.0,81.0,102.0,12-81-102,divergente
